In [1]:
from flask import Flask, render_template_string, request, redirect, session, Response, jsonify, send_from_directory
from pymongo import MongoClient
import cv2
import mediapipe as mp
import numpy as np
import pickle
import os
import threading
import time
import base64
from werkzeug.utils import secure_filename
from datetime import datetime
from bson import ObjectId
import random

app = Flask(__name__)
app.secret_key = 'your_secret_key'
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['MAX_CONTENT_LENGTH'] = 100 * 1024 * 1024  # 100MB max upload size

# MongoDB Setup
client = MongoClient('mongodb://localhost:27017/')
db = client['pose_app']
users_col = db['users']
history_col = db['history']

# Create directories if they don't exist
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs('saved_models', exist_ok=True)

# Load MediaPipe pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Global variables for video streaming
output_frame = None
lock = threading.Lock()
stop_event = threading.Event()
current_analysis_id = None
frame_rate = 60  # Target frame rate for smoother video
current_suggestion = ""
current_prediction = None
current_confidence = None

# Exercise-specific suggestions
EXERCISE_SUGGESTIONS = {
    'squat': {
        'Correct': ["Great form! Keep your back straight.", "Perfect squat depth!"],
        'Incorrect': ["Keep your knees behind your toes.", "Maintain a straight back.", "Go deeper into your squat."],
        'No pose': ["Make sure your whole body is visible."]
    },
    'deadlift': {
        'Correct': ["Excellent form! Keep your back neutral.", "Good hip hinge movement!"],
        'Incorrect': ["Keep your back straight, don't round it.", "Engage your core more.", "Drive through your heels."],
        'No pose': ["Step back to show your full body."]
    },
    'overhead_press': {
        'Correct': ["Good arm alignment!", "Great core engagement!"],
        'Incorrect': ["Keep your elbows slightly forward.", "Don't arch your back too much.", "Press directly overhead."],
        'No pose': ["Move back to show your full posture."]
    }
}

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

def get_random_suggestion(exercise, feedback):
    suggestions = EXERCISE_SUGGESTIONS.get(exercise, {}).get(feedback, ["Keep practicing!"])
    return random.choice(suggestions)

class ExerciseFormInference:
    def __init__(self, model_path):
        with open(model_path, 'rb') as f:
            data = pickle.load(f)
        self.model = data['model']
        self.scaler = data['scaler']
        self.prev_landmarks = None

    def extract_features_from_landmarks(self, landmarks, exercise_name):
        joint_coordinates = np.array([[lm.x, lm.y, lm.z] for lm in landmarks]).flatten()
        
        right_knee_angle = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])
        
        left_knee_angle = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
        
        right_hip_angle = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y])
        
        left_hip_angle = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y])
        
        right_elbow_angle = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y])
        
        left_elbow_angle = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
        
        right_shoulder_angle = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y])
        
        left_shoulder_angle = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
        
        left_shoulder_xy = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y])
        right_shoulder_xy = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y])
        left_hip_xy = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
        right_hip_xy = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y])
        shoulder_midpoint = (left_shoulder_xy + right_shoulder_xy) / 2
        hip_midpoint = (left_hip_xy + right_hip_xy) / 2
        vertical_vector = np.array([0, -1])
        torso_vector = hip_midpoint - shoulder_midpoint
        torso_angle_rad = np.arccos(np.clip(np.dot(torso_vector, vertical_vector) / (np.linalg.norm(torso_vector) * np.linalg.norm(vertical_vector) + 1e-6), -1.0, 1.0))
        torso_lean_angle = np.degrees(torso_angle_rad)
        
        depth_of_squat = 0
        if exercise_name == 'squat':
            depth_of_squat = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y - landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
        
        center_of_mass_x = np.mean([landmarks[idx].x for idx in [
            mp_pose.PoseLandmark.LEFT_SHOULDER.value,
            mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
            mp_pose.PoseLandmark.LEFT_HIP.value,
            mp_pose.PoseLandmark.RIGHT_HIP.value
        ]])
        center_of_mass_y = np.mean([landmarks[idx].y for idx in [
            mp_pose.PoseLandmark.LEFT_SHOULDER.value,
            mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
            mp_pose.PoseLandmark.LEFT_HIP.value,
            mp_pose.PoseLandmark.RIGHT_HIP.value
        ]])
        
        knee_diff = abs(right_knee_angle - left_knee_angle)
        hip_diff = abs(right_hip_angle - left_hip_angle)
        elbow_diff = abs(right_elbow_angle - left_elbow_angle)
        shoulder_diff = abs(right_shoulder_angle - left_shoulder_angle)
        
        frame_diff = 0
        if self.prev_landmarks is not None:
            current = np.array([[lm.x, lm.y, lm.z] for lm in landmarks])
            previous = np.array([[lm.x, lm.y, lm.z] for lm in self.prev_landmarks])
            frame_diff = np.linalg.norm(current - previous)
        self.prev_landmarks = landmarks
        
        features_arr = np.concatenate([
            joint_coordinates,
            [
                right_knee_angle, left_knee_angle, right_hip_angle, left_hip_angle,
                right_elbow_angle, left_elbow_angle, right_shoulder_angle, left_shoulder_angle,
                torso_lean_angle, depth_of_squat,
                center_of_mass_x, center_of_mass_y,
                knee_diff, hip_diff, elbow_diff, shoulder_diff,
                frame_diff
            ]
        ])
        return features_arr

    def predict(self, frame, exercise_name):
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        results = pose.process(image)
        pose.close()
        
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            features = self.extract_features_from_landmarks(results.pose_landmarks.landmark, exercise_name)
            features_scaled = self.scaler.transform([features])
            pred = self.model.predict(features_scaled)[0]
            conf = max(self.model.predict_proba(features_scaled)[0])
            return pred, conf, frame
        else:
            self.prev_landmarks = None
            return None, None, frame

def generate_frames(source=0, exercise='squat', model_name='Random Forest grid', username=None):
    global current_analysis_id, frame_rate, current_suggestion, current_prediction, current_confidence
    
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        print("Error: Could not open video source")
        return
    
    # Set higher resolution and FPS for smoother video
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, frame_rate)
    
    # Convert display name to filename format
    model_suffix = model_name.replace(" grid", "_grid").replace(" random", "_random")
    model_file = f"{exercise}_{model_suffix}.pkl"
    model_path = os.path.join("saved_models", model_file)
    
    if not os.path.exists(model_path):
        print(f"Model not found: {model_path}")
        return
    
    inferencer = ExerciseFormInference(model_path)
    global output_frame, lock, stop_event
    stop_event.clear()
    
    # Create a new analysis record
    analysis_record = {
        'username': username,
        'exercise': exercise,
        'model': model_name,
        'mode': 'live' if source == 0 else 'video',
        'start_time': datetime.now(),
        'end_time': None,
        'feedback': [],
        'average_confidence': 0.0,
        'correct_count': 0,
        'incorrect_count': 0,
        'total_frames': 0,
        'duration': 0.0,
        'suggestions': []
    }
    current_analysis_id = history_col.insert_one(analysis_record).inserted_id
    
    total_frames = 0
    correct_frames = 0
    confidence_sum = 0.0
    start_time = time.time()
    
    while cap.isOpened() and not stop_event.is_set():
        ret, frame = cap.read()
        if not ret:
            break

        if source == 0:
            frame = cv2.flip(frame, 1)

        # Resize frame for better performance while maintaining aspect ratio
        frame = cv2.resize(frame, (960, 540))  # Reduced resolution for smoother streaming
        
        # Get prediction
        pred, conf, frame = inferencer.predict(frame, exercise)
        
        if pred is not None:
            label = "Correct" if pred == 1 else "Incorrect"
            color = (0, 255, 0) if pred == 1 else (0, 0, 255)
            
            # Update global variables
            current_prediction = label
            current_confidence = f"{conf * 100:.1f}"
            
            # Get and store suggestion
            suggestion = get_random_suggestion(exercise, label)
            current_suggestion = suggestion
            
            # Add prediction text with suggestion
            cv2.putText(frame, f"{label} ({conf * 100:.1f}%)", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            cv2.putText(frame, f"Suggestion: {suggestion}", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            # Update analysis stats
            total_frames += 1
            confidence_sum += conf
            if pred == 1:
                correct_frames += 1
            
        else:
            suggestion = get_random_suggestion(exercise, "No pose")
            current_suggestion = suggestion
            current_prediction = "No pose"
            current_confidence = "0.0"
            cv2.putText(frame, "No pose detected", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            cv2.putText(frame, f"Suggestion: {suggestion}", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Encode frame as JPEG with optimized quality for better performance
        ret, buffer = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 85])
        frame_bytes = buffer.tobytes()

        with lock:
            output_frame = frame_bytes

        # Control frame rate to maintain smooth video
        elapsed_time = time.time() - start_time
        expected_time = total_frames / frame_rate
        if elapsed_time < expected_time:
            time.sleep(expected_time - elapsed_time)

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

    # Update the analysis record when finished
    if total_frames > 0:
        avg_confidence = (confidence_sum / total_frames) * 100
        correct_percentage = (correct_frames / total_frames) * 100
        duration = time.time() - start_time
        
        history_col.update_one(
            {'_id': current_analysis_id},
            {'$set': {
                'end_time': datetime.now(),
                'average_confidence': f"{avg_confidence:.1f}",
                'correct_percentage': f"{correct_percentage:.1f}",
                'total_frames': total_frames,
                'duration': duration,
                'suggestions': [current_suggestion] if current_suggestion else []
            }}
        )
    
    cap.release()
    current_analysis_id = None

@app.route('/')
def landing():
    if 'username' in session:
        return redirect('/main')
    return render_template_string(LANDING_HTML)

@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        if users_col.find_one({'username': username}):
            return render_template_string(SIGNUP_HTML, error="Username already exists. Try another.")

        users_col.insert_one({'username': username, 'password': password})
        return redirect('/login')

    return render_template_string(SIGNUP_HTML)

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        user = users_col.find_one({'username': username, 'password': password})
        if user:
            session['username'] = username
            return redirect('/main')
        else:
            return render_template_string(LOGIN_HTML, error="Invalid credentials.")

    return render_template_string(LOGIN_HTML)

@app.route('/logout')
def logout():
    session.clear()
    return redirect('/')

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

@app.route('/main')
def main():
    if 'username' not in session:
        return redirect('/login')
    
    model_dir = "saved_models"
    exercises = {
        'squat': [],
        'deadlift': [],
        'overhead_press': []
    }
    
    for exercise in exercises.keys():
        models = [f for f in os.listdir(model_dir) if f.startswith(exercise) and f.endswith(".pkl")]
        exercises[exercise] = [m.replace(f"{exercise}_", "").replace(".pkl", "").replace("_random", " random").replace("_grid", " grid") for m in models]
    
    return render_template_string(MAIN_HTML, username=session['username'], exercises=exercises)

@app.route('/video_feed')
def video_feed():
    if 'username' not in session:
        return "Unauthorized", 401

    exercise = request.args.get('exercise')
    mode = request.args.get('mode')
    model_name = request.args.get('model')

    if mode == 'live':
        source = 0
    elif mode == 'video':
        video_path = session.get('uploaded_video_path')
        if not video_path:
            return "No video uploaded", 400
        source = video_path
    else:
        return "Invalid mode", 400

    return Response(generate_frames(source, exercise, model_name, session['username']),
                   mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/upload_file', methods=['POST'])
def upload_file():
    if 'username' not in session:
        return jsonify({'error': 'Unauthorized'}), 401

    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
        
    file = request.files['file']
    mode = request.form.get('mode')

    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:
        filename = secure_filename(f"{session['username']}_{int(time.time())}_{file.filename}")
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        if mode == 'video':
            session['uploaded_video_path'] = filepath
        elif mode == 'image':
            session['uploaded_image_path'] = filepath

        return jsonify({'success': True, 'filename': filename, 'file_url': f'/uploads/{filename}'})

    return jsonify({'error': 'File upload failed'}), 400

@app.route('/analyze_image', methods=['POST'])
def analyze_image():
    if 'username' not in session:
        return jsonify({'error': 'Unauthorized'}), 401

    image_path = session.get('uploaded_image_path')
    exercise = request.form.get('exercise')
    model_name = request.form.get('model')

    if not image_path or not os.path.exists(image_path):
        return jsonify({'error': 'No image uploaded or file not found'}), 400

    model_suffix = model_name.replace(" grid", "_grid").replace(" random", "_random")
    model_file = f"{exercise}_{model_suffix}.pkl"
    model_path = os.path.join("saved_models", model_file)
    
    if not os.path.exists(model_path):
        return jsonify({'error': f'Model not found: {model_path}'}), 400

    image = cv2.imread(image_path)
    if image is None:
        return jsonify({'error': 'Failed to read image'}), 400
    
    image = cv2.resize(image, (1280, 720))
    inferencer = ExerciseFormInference(model_path)
    pred, conf, annotated_image = inferencer.predict(image, exercise)

    if pred is None:
        feedback = "No pose detected"
        confidence = None
        suggestion = get_random_suggestion(exercise, "No pose")
    else:
        feedback = "Correct" if pred == 1 else "Incorrect"
        confidence = f"{conf * 100:.1f}%"
        suggestion = get_random_suggestion(exercise, feedback)

    if pred is not None:
        color = (0, 255, 0) if pred == 1 else (0, 0, 255)
        cv2.putText(annotated_image, f"Prediction: {feedback} ({confidence})", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.putText(annotated_image, f"Suggestion: {suggestion}", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    _, buffer = cv2.imencode('.jpg', annotated_image)
    img_str = base64.b64encode(buffer).decode('utf-8')

    history_col.insert_one({
        'username': session['username'],
        'exercise': exercise,
        'model': model_name,
        'mode': 'image',
        'feedback': feedback,
        'confidence': confidence,
        'start_time': datetime.now(),
        'correct_percentage': float(confidence.strip('%')) if confidence else None,
        'average_confidence': float(confidence.strip('%')) if confidence else None,
        'suggestions': [suggestion] if suggestion else []
    })

    return jsonify({
        'feedback': feedback,
        'confidence': confidence,
        'suggestion': suggestion,
        'image': img_str
    })

@app.route('/stop_analysis', methods=['POST'])
def stop_analysis():
    global current_suggestion, current_prediction, current_confidence
    stop_event.set()
    
    return jsonify({
        'success': True,
        'suggestion': current_suggestion,
        'prediction': current_prediction if current_prediction else "No pose detected",
        'confidence': current_confidence if current_confidence else "0.0"
    })

@app.route('/history')
def history():
    if 'username' not in session:
        return redirect('/login')

    user_history = list(history_col.find({'username': session['username']}).sort('start_time', -1).limit(50))
    
    processed_history = []
    for h in user_history:
        # Convert ObjectId to string
        h['_id'] = str(h['_id'])
        
        # Handle start_time
        h['start_time'] = h.get('start_time', datetime.now())
        
        # Handle end_time and calculate duration
        if 'end_time' in h and h['end_time']:
            try:
                duration = (h['end_time'] - h['start_time']).total_seconds()
                h['duration'] = duration
            except:
                h['duration'] = None
        else:
            h['duration'] = None
        
        # Convert percentage strings to floats if needed
        if 'correct_percentage' in h and h['correct_percentage']:
            if isinstance(h['correct_percentage'], str):
                h['correct_percentage'] = float(h['correct_percentage'].rstrip('%'))
        else:
            h['correct_percentage'] = None
        
        # Ensure average_confidence is a float if it exists
        if 'average_confidence' in h and h['average_confidence']:
            if isinstance(h['average_confidence'], str):
                h['average_confidence'] = float(h['average_confidence'].rstrip('%'))
        else:
            h['average_confidence'] = None
        
        # Get first suggestion if available
        h['suggestion'] = h.get('suggestions', [None])[0] if h.get('suggestions') else None
        
        processed_history.append(h)

    return render_template_string(HISTORY_HTML, history=processed_history, username=session['username'])

# HTML Templates
LANDING_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>PosePerfect | Welcome</title>
    <style>
        :root {
            --primary-bg: #121212;
            --secondary-bg: #1e1e1e;
            --accent: #ff6b35;
            --accent-light: #ff8c5a;
            --text: #f5f5f5;
            --text-secondary: #b0b0b0;
        }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background-color: var(--primary-bg);
            color: var(--text);
            display: flex;
            flex-direction: column;
            min-height: 100vh;
        }
        .container {
            max-width: 800px;
            margin: 0 auto;
            padding: 2rem;
            text-align: center;
            flex: 1;
        }
        .hero {
            background: linear-gradient(135deg, var(--primary-bg), var(--secondary-bg));
            padding: 4rem 2rem;
            border-radius: 10px;
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.3);
            margin-bottom: 2rem;
        }
        h1 {
            color: var(--accent);
            font-size: 2.5rem;
            margin-bottom: 1rem;
        }
        p {
            font-size: 1.1rem;
            color: var(--text-secondary);
            margin-bottom: 2rem;
            line-height: 1.6;
        }
        .btn {
            display: inline-block;
            padding: 0.8rem 1.8rem;
            margin: 0.5rem;
            background-color: var(--accent);
            color: white;
            text-decoration: none;
            border-radius: 50px;
            font-weight: bold;
            transition: all 0.3s ease;
            border: 2px solid var(--accent);
        }
        .btn:hover {
            background-color: transparent;
            color: var(--accent);
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(255, 107, 53, 0.3);
        }
        .btn-secondary {
            background-color: transparent;
            color: var(--accent);
        }
        .btn-secondary:hover {
            background-color: var(--accent);
            color: white;
        }
        .logo {
            font-size: 3rem;
            font-weight: bold;
            color: var(--accent);
            margin-bottom: 1rem;
            display: inline-block;
        }
        .features {
            display: flex;
            justify-content: space-around;
            flex-wrap: wrap;
            margin-top: 3rem;
        }
        .feature {
            flex: 1;
            min-width: 200px;
            margin: 1rem;
            padding: 1.5rem;
            background-color: var(--secondary-bg);
            border-radius: 8px;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.2);
        }
        .feature-icon {
            font-size: 2rem;
            color: var(--accent);
            margin-bottom: 1rem;
        }
        footer {
            background-color: var(--secondary-bg);
            color: var(--text-secondary);
            text-align: center;
            padding: 1rem;
            margin-top: auto;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="hero">
            <div class="logo">PosePerfect</div>
            <h1>Master Your Form With AI</h1>
            <p>Get real-time feedback on your exercise technique using our advanced pose analysis system.</p>
            <div>
                <a href="/login" class="btn">Login</a>
                <a href="/signup" class="btn btn-secondary">Sign Up</a>
            </div>
        </div>
        
        <div class="features">
            <div class="feature">
                <div class="feature-icon">🏋️</div>
                <h3>Exercise Analysis</h3>
                <p>Detailed feedback on your form for various exercises</p>
            </div>
            <div class="feature">
                <div class="feature-icon">📊</div>
                <h3>Progress Tracking</h3>
                <p>Monitor your improvement over time</p>
            </div>
            <div class="feature">
                <div class="feature-icon">🤖</div>
                <h3>AI-Powered</h3>
                <p>Advanced machine learning models</p>
            </div>
        </div>
    </div>
    <footer>
        &copy; 2025 PosePerfect. All rights reserved.
    </footer>
</body>
</html>
"""

SIGNUP_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>PosePerfect | Sign Up</title>
    <style>
        :root {
            --primary-bg: #121212;
            --secondary-bg: #1e1e1e;
            --accent: #ff6b35;
            --accent-light: #ff8c5a;
            --text: #f5f5f5;
            --text-secondary: #b0b0b0;
            --error: #ff4d4d;
        }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background-color: var(--primary-bg);
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            flex-direction: column;
        }
        .container {
            background: var(--secondary-bg);
            padding: 2.5rem;
            border-radius: 10px;
            box-shadow: 0 10px 30px rgba(0, 0, 0, 0.3);
            width: 100%;
            max-width: 450px;
            margin: 1rem;
        }
        .logo {
            text-align: center;
            font-size: 2rem;
            font-weight: bold;
            color: var(--accent);
            margin-bottom: 1.5rem;
        }
        h2 {
            color: var(--text);
            text-align: center;
            margin-bottom: 1.5rem;
        }
        .form-group {
            margin-bottom: 1.2rem;
        }
        label {
            display: block;
            margin-bottom: 0.5rem;
            color: var(--text-secondary);
            font-weight: 500;
        }
        input {
            width: 100%;
            padding: 0.8rem;
            background-color: #2a2a2a;
            border: 1px solid #3a3a3a;
            border-radius: 6px;
            color: var(--text);
            font-size: 1rem;
            transition: border 0.3s;
        }
        input:focus {
            outline: none;
            border-color: var(--accent);
        }
        button {
            width: 100%;
            padding: 0.8rem;
            margin: 1.2rem 0 0.5rem;
            background-color: var(--accent);
            color: white;
            border: none;
            border-radius: 6px;
            font-size: 1rem;
            font-weight: bold;
            cursor: pointer;
            transition: all 0.3s;
        }
        button:hover {
            background-color: var(--accent-light);
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(255, 107, 53, 0.3);
        }
        .login-link {
            text-align: center;
            margin-top: 1.5rem;
            color: var(--text-secondary);
        }
        a {
            color: var(--accent);
            text-decoration: none;
            font-weight: 500;
        }
        a:hover {
            text-decoration: underline;
        }
        .error {
            color: var(--error);
            font-size: 0.9rem;
            margin-top: 0.3rem;
            display: block;
        }
        footer {
            background-color: var(--secondary-bg);
            color: var(--text-secondary);
            text-align: center;
            padding: 1rem;
            margin-top: auto;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="logo">PosePerfect</div>
        <h2>Create Your Account</h2>
        <form method="POST">
            <div class="form-group">
                <label for="username">Username</label>
                <input type="text" id="username" name="username" required />
            </div>
            <div class="form-group">
                <label for="password">Password</label>
                <input type="password" id="password" name="password" required />
            </div>
            {% if error %}
            <div class="error">{{ error }}</div>
            {% endif %}
            <button type="submit">Sign Up</button>
        </form>
        <div class="login-link">
            Already have an account? <a href="/login">Login here</a>
        </div>
    </div>
    <footer>
        &copy; 2025 PosePerfect. All rights reserved.
    </footer>
</body>
</html>
"""

LOGIN_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>PosePerfect | Login</title>
    <style>
        :root {
            --primary-bg: #121212;
            --secondary-bg: #1e1e1e;
            --accent: #ff6b35;
            --accent-light: #ff8c5a;
            --text: #f5f5f5;
            --text-secondary: #b0b0b0;
            --error: #ff4d4d;
        }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background-color: var(--primary-bg);
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            flex-direction: column;
        }
        .container {
            background: var(--secondary-bg);
            padding: 2.5rem;
            border-radius: 10px;
            box-shadow: 0 10px 30px rgba(0, 0, 0, 0.3);
            width: 100%;
            max-width: 450px;
            margin: 1rem;
        }
        .logo {
            text-align: center;
            font-size: 2rem;
            font-weight: bold;
            color: var(--accent);
            margin-bottom: 1.5rem;
        }
        h2 {
            color: var(--text);
            text-align: center;
            margin-bottom: 1.5rem;
        }
        .form-group {
            margin-bottom: 1.2rem;
        }
        label {
            display: block;
            margin-bottom: 0.5rem;
            color: var(--text-secondary);
            font-weight: 500;
        }
        input {
            width: 100%;
            padding: 0.8rem;
            background-color: #2a2a2a;
            border: 1px solid #3a3a3a;
            border-radius: 6px;
            color: var(--text);
            font-size: 1rem;
            transition: border 0.3s;
        }
        input:focus {
            outline: none;
            border-color: var(--accent);
        }
        button {
            width: 100%;
            padding: 0.8rem;
            margin: 1.2rem 0 0.5rem;
            background-color: var(--accent);
            color: white;
            border: none;
            border-radius: 6px;
            font-size: 1rem;
            font-weight: bold;
            cursor: pointer;
            transition: all 0.3s;
        }
        button:hover {
            background-color: var(--accent-light);
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(255, 107, 53, 0.3);
        }
        .signup-link {
            text-align: center;
            margin-top: 1.5rem;
            color: var(--text-secondary);
        }
        a {
            color: var(--accent);
            text-decoration: none;
            font-weight: 500;
        }
        a:hover {
            text-decoration: underline;
        }
        .error {
            color: var(--error);
            font-size: 0.9rem;
            margin-top: 0.3rem;
            display: block;
        }
       
        footer {
            background-color: var(--secondary-bg);
            color: var(--text-secondary);
            text-align: center;
            padding: 1rem;
            margin-top: auto;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="logo">PosePerfect</div>
        <h2>Welcome Back</h2>
        <form method="POST">
            <div class="form-group">
                <label for="username">Username</label>
                <input type="text" id="username" name="username" required />
            </div>
            <div class="form-group">
                <label for="password">Password</label>
                <input type="password" id="password" name="password" required />
            </div>
            {% if error %}
            <div class="error">{{ error }}</div>
            {% endif %}
            <button type="submit">Login</button>
        </form>
        <div class="signup-link">
            Don't have an account? <a href="/signup">Sign up here</a>
        </div>
    </div>
    <footer>
        &copy; 2025 PosePerfect. All rights reserved.
    </footer>
</body>
</html>
"""

MAIN_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>PosePerfect | Dashboard</title>
    <style>
        :root {
            --primary-bg: #121212;
            --secondary-bg: #1e1e1e;
            --accent: #ff6b35;
            --accent-light: #ff8c5a;
            --text: #f5f5f5;
            --text-secondary: #b0b0b0;
            --success: #4CAF50;
            --warning: #FFC107;
            --danger: #F44336;
        }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background-color: var(--primary-bg);
            color: var(--text);
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
        }
        header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 15px 0;
            border-bottom: 1px solid #333;
            margin-bottom: 30px;
        }
        .logo {
            font-size: 1.8rem;
            font-weight: bold;
            color: var(--accent);
        }
        .user-nav {
            display: flex;
            align-items: center;
            gap: 20px;
        }
        .user-nav a, .user-nav span {
            color: var(--text);
            text-decoration: none;
            font-weight: 500;
            transition: color 0.3s;
        }
        .user-nav a:hover {
            color: var(--accent);
        }
        .nav-btn {
            padding: 8px 16px;
            border-radius: 20px;
            background-color: var(--accent);
            color: white;
            font-weight: bold;
        }
        .nav-btn:hover {
            background-color: var(--accent-light);
            color: white;
        }
        .analysis-form {
            background: var(--secondary-bg);
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.2);
            margin-bottom: 30px;
        }
        .form-title {
            color: var(--accent);
            margin-bottom: 20px;
            font-size: 1.4rem;
        }
        .form-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 20px;
            margin-bottom: 20px;
        }
        .form-group {
            margin-bottom: 15px;
        }
        label {
            display: block;
            margin-bottom: 8px;
            color: var(--text-secondary);
            font-weight: 500;
        }
        select, input {
            width: 100%;
            padding: 12px;
            background-color: #2a2a2a;
            border: 1px solid #3a3a3a;
            border-radius: 6px;
            color: var(--text);
            font-size: 1rem;
            transition: all 0.3s;
        }
        select:focus, input:focus {
            outline: none;
            border-color: var(--accent);
            box-shadow: 0 0 0 2px rgba(255, 107, 53, 0.2);
        }
        .btn-group {
            display: flex;
            gap: 15px;
            margin-top: 20px;
        }
        button {
            padding: 12px 24px;
            border: none;
            border-radius: 6px;
            font-size: 1rem;
            font-weight: bold;
            cursor: pointer;
            transition: all 0.3s;
            flex: 1;
        }
        .btn-primary {
            background-color: var(--accent);
            color: white;
        }
        .btn-primary:hover {
            background-color: var(--accent-light);
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(255, 107, 53, 0.3);
        }
        .btn-danger {
            background-color: var(--danger);
            color: white;
        }
        .btn-danger:hover {
            background-color: #e53935;
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(244, 67, 54, 0.3);
        }
        .hidden {
            display: none;
        }
        #video-container {
            background: #000;
            border-radius: 10px;
            overflow: hidden;
            margin-bottom: 30px;
            position: relative;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.3);
        }
        #video-feed {
            width: 100%;
            display: block;
        }
        #feedback {
            background: var(--secondary-bg);
            padding: 20px;
            border-radius: 10px;
            margin-bottom: 30px;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.2);
        }
        .feedback-correct {
            border-left: 5px solid var(--success);
        }
        .feedback-incorrect {
            border-left: 5px solid var(--danger);
        }
        .feedback-neutral {
            border-left: 5px solid var(--warning);
        }
        .feedback-title {
            font-size: 1.2rem;
            margin-bottom: 10px;
            font-weight: bold;
        }
        .feedback-text {
            font-size: 1rem;
            color: var(--text-secondary);
        }
        #image-result {
            text-align: center;
            margin: 30px 0;
        }
        #image-result img {
            max-width: 100%;
            border-radius: 10px;
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.3);
        }
        .progress-container {
            width: 100%;
            background-color: #2a2a2a;
            border-radius: 6px;
            margin: 15px 0;
            overflow: hidden;
        }
        .progress-bar {
            height: 10px;
            background-color: var(--accent);
            border-radius: 6px;
            width: 0%;
            transition: width 0.3s;
        }
        .upload-status {
            font-size: 0.9rem;
            color: var(--text-secondary);
            margin-bottom: 15px;
        }
        .file-preview {
            margin-top: 15px;
            padding: 10px;
            background-color: #2a2a2a;
            border-radius: 6px;
            display: flex;
            align-items: center;
            gap: 10px;
        }
        .file-preview img {
            width: 50px;
            height: 50px;
            object-fit: cover;
            border-radius: 4px;
        }
        .file-info {
            flex: 1;
        }
        .file-name {
            font-weight: 500;
            margin-bottom: 5px;
        }
        .file-size {
            font-size: 0.8rem;
            color: var(--text-secondary);
        }
        
        #video-container {
            position: relative;
        }
        
        .suggestion-box {
            position: absolute;
            top: 10px;
            left: 10px;
            right: 10px;
            background-color: rgba(30, 30, 30, 0.8);
            padding: 10px;
            border-radius: 5px;
            color: white;
            z-index: 10;
        }
        
        .final-feedback {
            position: static;
            margin-top: 20px;
            background-color: var(--secondary-bg);
        }
        
        .prediction-text {
            font-weight: bold;
            font-size: 1.2em;
            margin-bottom: 5px;
        }
        
        .suggestion-text {
            font-size: 0.9em;
        }
        
        /* Add smooth transitions for video */
        #video-feed {
            transition: opacity 0.3s;
        }
        footer {
            background-color: var(--secondary-bg);
            color: var(--text-secondary);
            text-align: center;
            padding: 1rem;
            margin-top: auto;
        }
    </style>
</head>
<body>
    <div class="container">
        <header>
            <div class="logo">PosePerfect</div>
            <div class="user-nav">
                <span>Welcome, {{ username }}!</span>
                <a href="/history">History</a>
                <a href="/logout" class="nav-btn">Logout</a>
            </div>
        </header>

        <div class="analysis-form">
            <h2 class="form-title">Exercise Analysis</h2>
            <form id="analysis-form">
                <div class="form-grid">
                    <div class="form-group">
                        <label for="exercise">Exercise Type</label>
                        <select id="exercise" name="exercise" required>
                            <option value="">-- Select Exercise --</option>
                            {% for exercise, models in exercises.items() %}
                                <option value="{{ exercise }}">{{ exercise.replace('_', ' ').title() }}</option>
                            {% endfor %}
                        </select>
                    </div>

                    <div class="form-group">
                        <label for="mode">Analysis Mode</label>
                        <select id="mode" name="mode" required>
                            <option value="">-- Select Mode --</option>
                            <option value="live">Live Camera</option>
                            <option value="video">Video File</option>
                            <option value="image">Image File</option>
                        </select>
                    </div>

                    <div class="form-group">
                        <label for="model">Analysis Model</label>
                        <select id="model" name="model" required>
                            <option value="">-- Select Model --</option>
                        </select>
                    </div>
                </div>

                <div id="file-input-container" class="hidden">
                    <div class="form-group">
                        <label for="file-input">Upload File</label>
                        <input type="file" id="file-input" name="file" accept="video/*,image/*" />
                    </div>
                    <div id="upload-progress" class="hidden">
                        <div class="progress-container">
                            <div id="progress-bar" class="progress-bar"></div>
                        </div>
                        <div id="upload-status" class="upload-status">Uploading: 0%</div>
                    </div>
                    <div id="file-preview" class="file-preview hidden">
                        <img id="file-thumbnail" src="" alt="File thumbnail" />
                        <div class="file-info">
                            <div id="file-name" class="file-name"></div>
                            <div id="file-size" class="file-size"></div>
                        </div>
                    </div>
                </div>

                <div class="btn-group">
                    <button type="submit" class="btn-primary">Start Analysis</button>
                    <button type="button" id="stop-btn" class="btn-danger hidden">Stop Analysis</button>
                </div>
            </form>
        </div>
    
    <div id="video-container">
        <div id="suggestion-box" class="suggestion-box hidden">
            <div id="prediction-text" class="prediction-text"></div>
            <div id="suggestion-text" class="suggestion-text"></div>
        </div>
        <img id="video-feed" src="" alt="Video Feed"/>
    </div>
    
    <div id="feedback"></div>
        <div id="image-result"></div>
    </div>

    <script>
        const exercises = {{ exercises|tojson }};
        const modeSelect = document.getElementById('mode');
        const exerciseSelect = document.getElementById('exercise');
        const modelSelect = document.getElementById('model');
        const fileInputContainer = document.getElementById('file-input-container');
        const fileInput = document.getElementById('file-input');
        const videoFeed = document.getElementById('video-feed');
        const feedbackDiv = document.getElementById('feedback');
        const imageResultDiv = document.getElementById('image-result');
        const stopBtn = document.getElementById('stop-btn');
        const videoContainer = document.getElementById('video-container');
        const uploadProgress = document.getElementById('upload-progress');
        const progressBar = document.getElementById('progress-bar');
        const uploadStatus = document.getElementById('upload-status');
        const filePreview = document.getElementById('file-preview');
        const fileThumbnail = document.getElementById('file-thumbnail');
        const fileName = document.getElementById('file-name');
        const fileSize = document.getElementById('file-size');
        const suggestionBox = document.getElementById('suggestion-box');
        const predictionText = document.getElementById('prediction-text');
        const suggestionText = document.getElementById('suggestion-text');
        
        let analysisActive = false;

        // Update models dropdown when exercise changes
        exerciseSelect.addEventListener('change', () => {
            const exercise = exerciseSelect.value;
            modelSelect.innerHTML = '<option value="">-- Select Model --</option>';
            
            if (exercise && exercises[exercise]) {
                exercises[exercise].forEach(model => {
                    const option = document.createElement('option');
                    option.value = model;
                    option.textContent = model;
                    modelSelect.appendChild(option);
                });
            }
        });

        // Show/hide file input based on mode
        modeSelect.addEventListener('change', () => {
            if (modeSelect.value === 'video' || modeSelect.value === 'image') {
                fileInputContainer.classList.remove('hidden');
            } else {
                fileInputContainer.classList.add('hidden');
                imageResultDiv.innerHTML = '';
                filePreview.classList.add('hidden');
            }
            
            // Adjust video container visibility
            if (modeSelect.value === 'image') {
                videoContainer.classList.add('hidden');
            } else {
                videoContainer.classList.remove('hidden');
            }
        });

        // Handle file selection
        fileInput.addEventListener('change', (e) => {
            if (fileInput.files.length > 0) {
                const file = fileInput.files[0];
                fileName.textContent = file.name;
                fileSize.textContent = formatFileSize(file.size);
                
                // Create thumbnail for images
                if (file.type.startsWith('image/')) {
                    const reader = new FileReader();
                    reader.onload = (e) => {
                        fileThumbnail.src = e.target.result;
                    };
                    reader.readAsDataURL(file);
                } else {
                    fileThumbnail.src = '/static/video-icon.png'; // Placeholder for videos
                }
                
                filePreview.classList.remove('hidden');
            } else {
                filePreview.classList.add('hidden');
            }
        });

        function formatFileSize(bytes) {
            if (bytes === 0) return '0 Bytes';
            const k = 1024;
            const sizes = ['Bytes', 'KB', 'MB', 'GB'];
            const i = Math.floor(Math.log(bytes) / Math.log(k));
            return parseFloat((bytes / Math.pow(k, i)).toFixed(2)) + ' ' + sizes[i];
        }

        // Form submission handler
        document.getElementById('analysis-form').addEventListener('submit', async (e) => {
            e.preventDefault();
            if (analysisActive) {
                alert('Please stop the current analysis first!');
                return;
            }

            const exercise = exerciseSelect.value;
            const mode = modeSelect.value;
            const model = modelSelect.value;

            if (!exercise || !mode || !model) {
                alert('Please select all options');
                return;
            }

            if ((mode === 'video' || mode === 'image') && fileInput.files.length === 0) {
                alert('Please upload a file');
                return;
            }

            feedbackDiv.innerHTML = '';
            imageResultDiv.innerHTML = '';
            videoFeed.src = '';
            stopBtn.classList.remove('hidden');
            suggestionBox.classList.remove('hidden');

            if (mode === 'live') {
                startLiveStream(exercise, model);
            } else {
                // Show upload progress
                uploadProgress.classList.remove('hidden');
                progressBar.style.width = '0%';
                uploadStatus.textContent = 'Uploading: 0%';
                
                // Upload file first
                const formData = new FormData();
                formData.append('file', fileInput.files[0]);
                formData.append('mode', mode);

                try {
                    const xhr = new XMLHttpRequest();
                    xhr.open('POST', '/upload_file', true);
                    
                    xhr.upload.onprogress = (e) => {
                        if (e.lengthComputable) {
                            const percentComplete = Math.round((e.loaded / e.total) * 100);
                            progressBar.style.width = percentComplete + '%';
                            uploadStatus.textContent = `Uploading: ${percentComplete}%`;
                        }
                    };
                    
                    xhr.onload = () => {
                        uploadProgress.classList.add('hidden');
                        if (xhr.status === 200) {
                            const data = JSON.parse(xhr.responseText);
                            
                            if (data.error) {
                                showFeedback('error', data.error);
                                stopBtn.classList.add('hidden');
                                suggestionBox.classList.add('hidden');
                                return;
                            }

                            if (mode === 'video') {
                                startLiveStream(exercise, model);
                            } else if (mode === 'image') {
                                analyzeImage(exercise, model);
                                stopBtn.classList.add('hidden');
                            }
                        } else {
                            showFeedback('error', 'Upload failed');
                            stopBtn.classList.add('hidden');
                            suggestionBox.classList.add('hidden');
                        }
                    };
                    
                    xhr.send(formData);
                } catch (error) {
                    console.error('Error:', error);
                    showFeedback('error', 'An error occurred during file upload');
                    stopBtn.classList.add('hidden');
                    suggestionBox.classList.add('hidden');
                }
            }
        });

        function showFeedback(type, message) {
            feedbackDiv.className = 'feedback-' + type;
            feedbackDiv.innerHTML = `
                <div class="feedback-title">${type.charAt(0).toUpperCase() + type.slice(1)} Form</div>
                <div class="feedback-text">${message}</div>
            `;
        }

        stopBtn.addEventListener('click', async () => {
            if (!analysisActive) return;
            try {
                const response = await fetch('/stop_analysis', {method: 'POST'});
                const data = await response.json();
                
                analysisActive = false;
                videoFeed.src = '';
                stopBtn.classList.add('hidden');
                
                if (data.success) {
                    // Display final feedback
                    if (data.prediction === 'Correct') {
                        showFeedback('correct', `Your form was correct. ${data.suggestion}`);
                    } else if (data.prediction === 'Incorrect') {
                        showFeedback('incorrect', `Your form needs work. ${data.suggestion}`);
                    } else {
                        showFeedback('neutral', data.suggestion);
                    }
                    
                    // Update the suggestion box with final feedback
                    updateSuggestion(
                        data.prediction || 'Analysis complete',
                        data.confidence || 'N/A',
                        data.suggestion || 'No suggestions available'
                    );
                    
                    // Change to static feedback style
                    suggestionBox.classList.add('final-feedback');
                } else {
                    showFeedback('error', 'Failed to stop analysis');
                    suggestionBox.classList.add('hidden');
                }
            } catch (error) {
                console.error('Error stopping analysis:', error);
                showFeedback('error', 'Failed to stop analysis');
                suggestionBox.classList.add('hidden');
            }
        });

        function startLiveStream(exercise, model) {
            analysisActive = true;
            const mode = modeSelect.value;
            videoFeed.src = `/video_feed?exercise=${exercise}&mode=${mode}&model=${encodeURIComponent(model)}`;
            showFeedback('neutral', 'Analysis started...');
        }

        async function analyzeImage(exercise, model) {
            showFeedback('neutral', 'Analyzing image...');
            
            try {
                const formData = new URLSearchParams();
                formData.append('exercise', exercise);
                formData.append('model', model);
                
                const res = await fetch('/analyze_image', {
                    method: 'POST',
                    headers: {'Content-Type': 'application/x-www-form-urlencoded'},
                    body: formData
                });
                
                const data = await res.json();
                
                if (data.error) {
                    showFeedback('error', data.error);
                    return;
                }
                
                if (data.feedback === 'Correct') {
                    showFeedback('correct', `Form is correct with ${data.confidence} confidence`);
                    updateSuggestion('Correct', data.confidence, data.suggestion);
                } else if (data.feedback === 'Incorrect') {
                    showFeedback('incorrect', `Form is incorrect with ${data.confidence} confidence`);
                    updateSuggestion('Incorrect', data.confidence, data.suggestion);
                } else {
                    showFeedback('neutral', data.feedback);
                    updateSuggestion('No pose', 'N/A', data.suggestion);
                }
                
                if (data.image) {
                    imageResultDiv.innerHTML = `<img src="data:image/jpeg;base64,${data.image}" />`;
                }
                
                suggestionBox.classList.remove('hidden');
                suggestionBox.classList.add('final-feedback');
            } catch (error) {
                console.error('Error analyzing image:', error);
                showFeedback('error', 'An error occurred during image analysis');
                suggestionBox.classList.add('hidden');
            }
        }
        
        function updateSuggestion(prediction, confidence, suggestion) {
            suggestionBox.classList.remove('hidden');
            
            predictionText.textContent = `${prediction}${confidence ? ` (${confidence})` : ''}`;
            suggestionText.textContent = `Suggestion: ${suggestion}`;
            
            // Set color based on prediction
            if (prediction.includes('Correct')) {
                predictionText.style.color = '#4CAF50';
            } else if (prediction.includes('Incorrect')) {
                predictionText.style.color = '#F44336';
            } else {
                predictionText.style.color = '#FFC107';
            }
        }
    </script>
    <footer>
        &copy; 2025 PosePerfect. All rights reserved.
    </footer>
</body>
</html>
"""

HISTORY_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>PosePerfect | History</title>
    <style>
        :root {
            --primary-bg: #121212;
            --secondary-bg: #1e1e1e;
            --accent: #ff6b35;
            --accent-light: #ff8c5a;
            --text: #f5f5f5;
            --text-secondary: #b0b0b0;
            --success: #4CAF50;
            --warning: #FFC107;
            --danger: #F44336;
        }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background-color: var(--primary-bg);
            color: var(--text);
            display: flex;
            flex-direction: column;
            min-height: 100vh;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            flex: 1;
        }
        header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 15px 0;
            border-bottom: 1px solid #333;
            margin-bottom: 30px;
        }
        .logo {
            font-size: 1.8rem;
            font-weight: bold;
            color: var(--accent);
        }
        .user-nav {
            display: flex;
            align-items: center;
            gap: 20px;
        }
        .user-nav a, .user-nav span {
            color: var(--text);
            text-decoration: none;
            font-weight: 500;
            transition: color 0.3s;
        }
        .user-nav a:hover {
            color: var(--accent);
        }
        .nav-btn {
            padding: 8px 16px;
            border-radius: 20px;
            background-color: var(--accent);
            color: white;
            font-weight: bold;
        }
        .nav-btn:hover {
            background-color: var(--accent-light);
            color: white;
        }
        .page-title {
            font-size: 1.8rem;
            margin-bottom: 20px;
            color: var(--accent);
        }
        
        .stats-container {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }
        .stat-card {
            background: var(--secondary-bg);
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.2);
            text-align: center;
        }
        .stat-value {
            font-size: 2rem;
            font-weight: bold;
            color: var(--accent);
            margin: 10px 0;
        }
        .stat-label {
            font-size: 0.9rem;
            color: var(--text-secondary);
        }
        
        .history-table {
            width: 100%;
            border-collapse: collapse;
            background: var(--secondary-bg);
            border-radius: 10px;
            overflow: hidden;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.2);
            margin-bottom: 30px;
        }
        .history-table th, 
        .history-table td {
            padding: 15px;
            text-align: left;
            border-bottom: 1px solid #333;
        }
        .history-table th {
            background-color: var(--accent);
            color: white;
            font-weight: bold;
        }
        .history-table tr:hover {
            background-color: #2a2a2a;
        }
        
        .status-correct {
            color: var(--success);
            font-weight: bold;
        }
        .status-incorrect {
            color: var(--danger);
            font-weight: bold;
        }
        .status-neutral {
            color: var(--warning);
            font-weight: bold;
        }
        
        .exercise-badge {
            display: inline-block;
            padding: 4px 10px;
            border-radius: 20px;
            font-size: 0.8rem;
            font-weight: bold;
            text-transform: uppercase;
        }
        .badge-squat {
            background-color: rgba(76, 175, 80, 0.2);
            color: var(--success);
        }
        .badge-deadlift {
            background-color: rgba(255, 193, 7, 0.2);
            color: var(--warning);
        }
        .badge-overhead {
            background-color: rgba(66, 165, 245, 0.2);
            color: #42a5f5;
        }
        
        .empty-history {
            text-align: center;
            padding: 60px 20px;
            color: var(--text-secondary);
        }
        .empty-icon {
            font-size: 3rem;
            margin-bottom: 20px;
            color: var(--accent);
        }
        .empty-message {
            font-size: 1.2rem;
            margin-bottom: 20px;
        }
        .empty-action {
            margin-top: 20px;
        }
        
        .suggestion-cell {
            max-width: 300px;
            white-space: nowrap;
            overflow: hidden;
            text-overflow: ellipsis;
        }
        
        footer {
            background-color: var(--secondary-bg);
            color: var(--text-secondary);
            text-align: center;
            padding: 1rem;
        }
    </style>
</head>
<body>
    <div class="container">
        <header>
            <div class="logo">PosePerfect</div>
            <div class="user-nav">
                <span>Welcome, {{ username }}!</span>
                <a href="/main">Back to Dashboard</a>
                <a href="/logout" class="nav-btn">Logout</a>
            </div>
        </header>

        <h1 class="page-title">Your Analysis History</h1>
        
        {% if history %}
        <div class="stats-container">
            <div class="stat-card">
                <div class="stat-value">{{ history|length }}</div>
                <div class="stat-label">Total Sessions</div>
            </div>
            <div class="stat-card">
                <div class="stat-value">
                    {% set correct_sessions = history|selectattr('correct_percentage', 'number')|list %}
                    {% if correct_sessions %}
                        {{ (correct_sessions|sum(attribute='correct_percentage') / correct_sessions|length)|round(1) }}%
                    {% else %}
                        N/A
                    {% endif %}
                </div>
                <div class="stat-label">Average Correct</div>
            </div>
            <div class="stat-card">
                <div class="stat-value">
                    {% set durations = history|selectattr('duration', 'number')|list %}
                    {% if durations %}
                        {{ (durations|sum(attribute='duration') / durations|length)|round(1) }}s
                    {% else %}
                        N/A
                    {% endif %}
                </div>
                <div class="stat-label">Average Duration</div>
            </div>
            <div class="stat-card">
                <div class="stat-value">
                    {% set exercises = history|groupby('exercise')|list %}
                    {{ exercises|length }}
                </div>
                <div class="stat-label">Different Exercises</div>
            </div>
        </div>

        <table class="history-table">
            <thead>
                <tr>
                    <th>Exercise</th>
                    <th>Date</th>
                    <th>Duration</th>
                    <th>Correct %</th>
                    <th>Confidence</th>
                    <th>Suggestion</th>
                </tr>
            </thead>
            <tbody>
                {% for record in history %}
                <tr>
                    <td>
                        <span class="exercise-badge badge-{{ record.exercise.split('_')[0] }}">
                            {{ record.exercise.replace('_', ' ').title() }}
                        </span>
                    </td>
                    <td>
                        {% if record.start_time %}
                            {{ record.start_time.strftime('%Y-%m-%d %H:%M') }}
                        {% else %}
                            N/A
                        {% endif %}
                    </td>
                    <td>
                        {% if record.duration is not none %}
                            {{ "%.1f"|format(record.duration) }} seconds
                        {% else %}
                            N/A
                        {% endif %}
                    </td>
                    <td class="{% if record.correct_percentage is not none and record.correct_percentage >= 70 %}status-correct{% elif record.correct_percentage is not none and record.correct_percentage >= 40 %}status-neutral{% else %}status-incorrect{% endif %}">
                        {% if record.correct_percentage is not none %}
                            {{ "%.1f"|format(record.correct_percentage) }}%
                        {% else %}
                            N/A
                        {% endif %}
                    </td>
                    <td>
                        {% if record.average_confidence is not none %}
                            {{ "%.1f"|format(record.average_confidence) }}%
                        {% else %}
                            N/A
                        {% endif %}
                    </td>
                    <td class="suggestion-cell" title="{{ record.suggestion if record.suggestion else 'No suggestion' }}">
                        {{ record.suggestion if record.suggestion else 'N/A' }}
                    </td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
        {% else %}
        <div class="empty-history">
            <div class="empty-icon">📊</div>
            <div class="empty-message">No analysis history yet</div>
            <p>Start analyzing your exercises to see your progress and history here.</p>
            <div class="empty-action">
                <a href="/main" class="nav-btn">Start Analyzing</a>
            </div>
        </div>
        {% endif %}
    </div>
    <footer>
        &copy; 2025 PosePerfect. All rights reserved.
    </footer>
</body>
</html>
"""

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Jul/2025 14:31:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2025 14:31:07] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Jul/2025 14:31:08] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2025 14:31:15] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [04/Jul/2025 14:31:15] "GET /main HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2025 14:31:48] "GET /static/video-icon.png HTTP/1.1" 404 -
127.0.0.1 - - [04/Jul/2025 14:31:49] "POST /upload_file HTTP/1.1" 200 -
D:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [04/Jul/2025 14:31:52] "GET /video_feed?exercise=squat&mode=video&model=Random%20Forest%20grid HTTP/1.1" 200 -
D:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\An

http://127.0.0.1:5000